First use: 190329 (CoNLL 2012)

Second use: 190502 (ANCOR)

Third use: 190531 (creating the firsts lists for both datasets)

In [1]:
import csv
import pandas as pd
import numpy as np
from os import listdir
from shutil import copyfile

In [2]:
data_source = "CoNLL_2012"
#data_source = "ANCORE"

# Prepare data for preprocessing

In [3]:
test_df = pd.read_csv("../../data/structured_{}/test.csv".format(data_source))
test_df.head()

,Unnamed: 0,col0,col0.1,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14
0,0,pt/nt/44/nt_4409,pt,0,0,In,IN,(TOP(S(PP*,-,-,-,Acts,*,-,O,O,O
1,1,pt/nt/44/nt_4409,pt,0,1,Jerusalem,NNP,(NP*)),-,-,-,Acts,*,(6),B,B-N,B-N
2,2,pt/nt/44/nt_4409,pt,0,2,Saul,NNP,(NP*),-,-,-,Acts,*,(9),B,B-N,B-N
3,3,pt/nt/44/nt_4409,pt,0,3,was,VBD,(VP*,-,-,-,Acts,*,-,O,O,O
4,4,pt/nt/44/nt_4409,pt,0,4,still,RB,(ADVP*),-,-,-,Acts,*,-,O,O,O


In [4]:
test_df.shape

(169579, 17)

In [5]:
sets = ["dev", "test", "train"] #my datasets labels
splits = ["valid", "test", "train"] #fairseq's datasets labels
sides = ["input", "label"]

In [3]:
for i in range(3): #split index
    
    split_dataframe = pd.read_csv("../../data/structured_{}/{}.csv".format(data_source, sets[i]))
    df_size = split_dataframe.shape[0]
    
    parent_path = "../../data/structured_{}_to_preprocess/{}.".format(data_source, splits[i])
    with open(parent_path + "input", "w") as f_input, open(parent_path + "label", "w") as f_label:

        k = 0
        mini_input = [split_dataframe.loc[k, "col3"]]
        mini_label = [split_dataframe.loc[k, "col12"]]

        while k < df_size - 1:
            #if end of a part or a document
            print(k)
            if (split_dataframe.loc[k, "col0"] != split_dataframe.loc[k+1, "col0"]) or \
            (split_dataframe.loc[k+1, "col2"] == 0) or \
            (split_dataframe.loc[k, "col1"] != split_dataframe.loc[k+1, "col1"]): #different doc or sentence or part ==> different "sentence" to translate or label

                f_input.write(" ".join(mini_input) + "\n")
                f_label.write(" ".join(mini_label) + "\n")

                mini_input = [split_dataframe.loc[k+1, "col3"]]
                mini_label = [split_dataframe.loc[k+1, "col12"]]
                
            else:
                mini_input.append(str(split_dataframe.loc[k+1, "col3"]))
                mini_label.append(str(split_dataframe.loc[k+1, "col12"]))

            k += 1
        
        #last line of dataframe
        mini_input.append(split_dataframe.loc[k, "col3"])
        mini_label.append(split_dataframe.loc[k, "col12"])
        f_input.write(" ".join(mini_input) + "\n")
        f_label.write(" ".join(mini_label) + "\n")

NameError: name 'sdf' is not defined

In [ ]:
num_lines = 0
with open("../../data/structured_{}_to_preprocess/train.input".format(data_source, sets[i]), 'r') as f:
    for line in f:
        num_lines += 1
print("Number of lines:")
print(num_lines)

# Save first sentences indices to separate documents

In [ ]:
firsts_list = [[], [], []]

for i in range(3): #split index
    
    split_dataframe = pd.read_csv("../../data/structured_{}/{}.csv".format(data_source, sets[i]))
    df_size = split_dataframe.shape[0]
    sentence_ctr = 0
    
    k = 0
    firsts_list[i].append(k)
    
    k += 1

    while k < df_size - 1:
        #no need to worry about last line, as no sentence is made of a single line
        
        #if end of sentence
        if (split_dataframe.loc[k+1, "col2"] == 0):
            sentence_ctr += 1
        
        #if end of a part or a document
        if (split_dataframe.loc[k, "col0"] != split_dataframe.loc[k+1, "col0"]) or \
        (split_dataframe.loc[k, "col1"] != split_dataframe.loc[k+1, "col1"]): #different doc or part ==> new first document line

            firsts_list[i].append(sentence_ctr)
        
        k += 1

In [ ]:
[len(e) for e in firsts_list]

In [ ]:
firsts_list[0]

In [ ]:
for i in range(3):
    with open("../../data/{}.firsts.txt".format(splits[i]), "w") as f:
        for first in firsts_list[i]:
            f.write("{}\n".format(first))

# Check documents, parts and sentences

In [ ]:
for i in range(3): #split index
    
    split_dataframe = pd.read_csv("../../data/structured_{}/{}.csv".format(data_source, sets[i]))
    df_size = split_dataframe.shape[0]

    k = 0
    print("new document")
    print("new part")
    print("new sentence")
    while k < df_size - 1:
        print(k)
        if (split_dataframe.loc[k, "col0"] != split_dataframe.loc[k+1, "col0"]): #END OF DOCUMENT + PART CONDITION
            print("new document")
            print("new part")
        elif (split_dataframe.loc[k, "col1"] != split_dataframe.loc[k+1, "col1"]): #END OF PART (NOT DOC) CONDITION
            print("new part")
            print("DAMN")
        if (split_dataframe.loc[k+1, "col2"] == 0): #END OF SENTENCE CONDITION
            print("new sentence")
        k += 1